# Basics
ARM supplies IPs such as processor cores, bus switches, NVIC etc.ARM also supplies processor IP that combines some of its other IPs: Cortex M4 has a processor core, Nested Vector Interrupt Controller (NVIC), a Memory Protection Unit (MPU). It also has several memeory, device interface buses, a Trace port interface, Debug port interface.We will use cortex M4 as the representative processor (It is a ARM-V7 architecture. The latest architecture is v8)

<pre>
<figcaption>Cortex M4 Overview</figcaption>
<img src="cortexm4_overview.webp" width=800/>

</pre>

<pre>
<figcaption>Cortex M4 Block diagram</figcaption>
<img src="cortexm4_blockdiagram.svg" width=800/>

</pre>

<pre>
<figcaption>Cortex M4 System address map</figcaption>
<img src="cortexm_systemAddressMap.png" width=1024/>

</pre>


The system address map shown is only the recommended map. User can change it (with some exceptions such as the addresses of private peripheral bus)



# Registers
The processor has 32-registers that includes 13 general-purpose registers and several special-purpose registers.

The processor has the following 32-bit registers:

  + 13 general-purpose registers, R0-R12.
  + Stack Pointer (SP), R13 alias of banked registers, SP_process and SP_main.
  + Link Register (LR), R14.
  + Program Counter (PC), R15.
  + Special-purpose Program Status Registers, (xPSR).

**Stack pointer**
Register R13 is used as the Stack Pointer (SP). Because the SP ignores writes to bits [1:0], it is autoaligned to a word, four-byte boundary.

Handler mode always uses SP_main, but you can configure Thread mode to use either SP_main or SP_process.

**Link register**
Register R14 is the subroutine Link Register (LR).

The LR receives the return address from PC when a Branch and Link (BL) or Branch and Link with Exchange (BLX) instruction is executed.

The LR is also used for exception return.

At all other times, you can treat R14 as a general-purpose register.

**Program counter**
Register R15 is the Program Counter (PC).

Bit [0] is always 0, so instructions are always aligned to word or halfword boundaries.

**xPSR**
It contains

## Stack
The stack is a RAM area that is reserved to be used during the execution of the program to temporarily store register values, return address (during subroutine calls), parameter values (during subroutine calls), return value, local variables of a subroutine etc. For ex., when a subroutine is called, ARM Procedure Call Standard requires that it is the subroutines job to preserve the state of registers *r4*-*r11* if it plans to use any of it. So the subroutine will push the registers it wants to use in *r4*-*r11* into the stack and then once it is done using them, it will pop the stack and restore the register values. Similarly, when the main routine calls the subroutine, it will use the *bl* instruction, which will automatically store the return address in the *LR* register. The subroutine hence first pushes the contents of *LR* into the stack and then, at the end of execution, pops the originally pushed stack content into *PC*. This is all captured in the below example:
(*Note: ip register is just an alias for r12. It is used as an Intra-Procedural scratch pad register and hence the name. Don't worry about it yet*)


In [ ]:
    .globl  main
    .extern abs
    .extern printf

    .text
output_str:
    .ascii     "The answer is %d\n\0"

    @ returns abs(z)+x+y
    @ r0 = x, r1 = y, r2 = z
    .align 4
do_something:
    push    {r4, lr}
    add     r4, r0, r1
    mov     r0, r2
    bl      abs
    add     r0, r4, r0
    pop     {r4, pc}

main:
    push    {ip, lr}
    mov     r0, #1
    mov     r1, #3
    mov     r2, #-4
    bl      do_something
    mov     r1, r0
    ldr     r0, =output_str
    bl      printf
    mov     r0, #0
    pop     {ip, pc}

Besides these uses, parameters passed to a sub-routine and its local variables can be stored in the stack and instructions can push/pop them using their relative locations from the *SP* register. The important point to note here is that, as the code executes and various subroutines are called and returned, the stack grows and wanes. Since this happens in runtime, it may not be possible to predict which functions may be called when, and hence, it is not possible to predict exactly how big the stack needs to be. So sometimes, the stack pointer goes out of the designated area into other memory areas (ex. heap) and this condition is called **stack overflow**

Main stack pointer vs. process stack pointer

# FIQ vs. IRQ
Faster Interrupt Request (FIQ) has higher priority than IRQ. It also doesn't require state saving into the stack because each FIQ has its own shadow registers.


# References
1. [ARM cortex reference manual](https://developer.arm.com/documentation/100166/0001)
2. [Flash Patch and Breakpoint unit](https://developer.arm.com/documentation/ddi0337/h/debug/about-the-flash-patch-and-breakpoint-unit--fpb-)